In [ ]:
%pip -q install google-adk
%pip -q install google-genai


In [ ]:
import os
import re
import urllib.parse
import warnings
from datetime import datetime

try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ModuleNotFoundError:
    pass

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

from IPython.display import display, Markdown, HTML

warnings.filterwarnings("ignore")

def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response.strip()


def maps_link(place: str, city: str) -> str:
    base = "https://www.google.com/maps/search/?api=1&query="
    return base + urllib.parse.quote_plus(f"{place} {city}")


def add_maps_links(markdown_text: str, city: str) -> str:
    def repl(match: re.Match) -> str:
        attraction = match.group(1).strip()
        url = maps_link(attraction, city)
        return f"- [{attraction}]({url})"
    pattern = r"^[\-\*]\s+(.+?)(?:\n|$)"
    return re.sub(pattern, repl, markdown_text, flags=re.MULTILINE)


def colapsar_por_dia(markdown_text: str) -> str:
    blocos = markdown_text.split("### Dia")
    html = ""
    for bloco in blocos[1:]:
        linhas = bloco.split("\n")
        dia_titulo = linhas[0].strip()
        corpo = "\n".join(linhas[1:]).strip()
        # Split the body by '-' and join with newline to force line breaks
        corpo_com_quebras = "\n".join(corpo.split('-'))
        html += (
            f"<details style='margin-bottom:0.8em'>"
            f"<summary style='font-weight:bold;font-size:1.1em'>Dia {dia_titulo}</summary>"
            # Use the modified corpo_com_quebras with line breaks
            f"<pre style='white-space:pre-wrap'>{corpo_com_quebras}</pre></details>"
        )
    return html

def agente_buscador(destino: str, data_de_hoje: str) -> str:
    agent = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Encontre até 5 roteiros de viagem recentes (até 1 mês) sobre o destino fornecido.
        Use fontes confiáveis e mostre sugestões com programação diária e dicas práticas.
        """,
        description="Busca roteiros recentes",
        tools=[google_search],
    )
    entrada = f"Destino: {destino}\nData atual: {data_de_hoje}"
    return call_agent(agent, entrada)


def agente_planejador(destino: str, preferencias: str, roteiro_base: str, pontos_desejados: str) -> str:
    agent = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Gere um plano de viagem personalizado com base nas preferências, pontos turísticos desejados e estilo de viagem.
        Use também os roteiros de referência encontrados.
        """,
        description="Planejador de viagem",
        tools=[google_search],
    )
    entrada = f"Destino: {destino}\nPreferências: {preferencias}\nPontos desejados: {pontos_desejados}\nBase: {roteiro_base}"
    return call_agent(agent, entrada)


def agente_documentos(destino: str, cidade_origem: str) -> str:
    agent = Agent(
        name="agente_documentos",
        model="gemini-2.0-flash",
        instruction="""
        Liste os documentos necessários para uma viagem internacional entre a cidade de origem e o destino.
        Inclua passaporte, visto, vacinas etc. Utilize busca em fontes confiáveis, Seja direto, sem muito texto.
        """,
        description="Documentação necessária",
        tools=[google_search],
    )
    entrada = f"Origem: {cidade_origem}\nDestino: {destino}"
    return call_agent(agent, entrada)


def agente_voo(destino: str, cidade_origem: str, data_ida: str, data_volta: str) -> str:
    agent = Agent(
        name="agente_voo",
        model="gemini-2.0-flash",
        instruction="""
        Pesquise o menor valor de passagens de ida e volta para o destino informado.
        Inclua a companhia aérea e horários aproximados. Use busca atual. Seja simples e direto, deixe a leitura
        boa e use emoji para topicos.
        """,
        description="Busca de passagens aéreas",
        tools=[google_search],
    )
    entrada = f"Origem: {cidade_origem}\nDestino: {destino}\nIda: {data_ida}\nVolta: {data_volta}"
    return call_agent(agent, entrada)


def agente_acomodacao(destino: str, data_ida: str, data_volta: str, orcamento_float: float) -> str:
    agent = Agent(
        name="agente_acomodacao",
        model="gemini-2.0-flash",
        instruction="""
        Sugira 3 hospedagens (hotel ou hostel ou Airbnb) para o destino e datas informadas, com valor até 40% do orçamento diário.
        Mostre nome, tipo, localização, preço por noite e link, Seja simples e sucinto.
        """,
        description="Sugestões de hospedagem",
        tools=[google_search],
    )
    entrada = f"Destino: {destino}\nData ida: {data_ida}\nData volta: {data_volta}\nOrçamento diário: {orcamento_float}"
    return call_agent(agent, entrada)


def obter_previsao_media(destino: str, data_inicio: str, data_fim: str) -> str:
    agent = Agent(
        name="agente_clima",
        model="gemini-2.0-flash",
        instruction="""
        Mostre o clima médio entre as datas da viagem. Se for internacional, inclua cotação da moeda local em reais.
        Use emojis e uma apresentação amigável. Seja sucinto e simples
        """,
        description="Clima e câmbio",
        tools=[google_search],
    )
    entrada = f"Destino: {destino}\nData início: {data_inicio}\nData fim: {data_fim}"
    return call_agent(agent, entrada)


def agente_estruturador(destino: str, roteiro_bruto: str) -> str:
    agent = Agent(
        name="agente_estruturador",
        model="gemini-2.0-flash",
        instruction="""
        Estruture o roteiro em sessões por dia (### Dia 1, Dia 2...).
        Use bullets com emoji, nome da atração e horário estimado.
        Não inclua URLs; elas serão adicionadas por pós-processamento.
        Mostre o custo total e uma dica no final.
        """,
        description="Organizador de roteiro final",
    )
    entrada = f"Destino: {destino}\nRoteiro: {roteiro_bruto}"
    return call_agent(agent, entrada)



# Início
print(r"""
             __|__
--@--@--@--@--(_)--@--@--@--@--

     ✈️  Bem-vindo à Luis Travel ✈️
     Seu roteiro começa aqui. 🌍
""")


print("✈️ Chega de perder horas pesquisando atrações, clima e hospedagem por conta própria!")
print("✨ Com apenas algumas perguntas, eu preparo o roteiro perfeito pra sua viagem \nrápido, personalizado e do jeitinho que você gosta.\n")
print("📋 Vamos começar? Me diga para onde você vai e eu cuido do resto. 😄")


cidade_origem = input("🏠 Cidade de origem: ")
destino = input("📍 Para qual cidade você vai viajar? ")
data_ida = input("🛫 Data de ida? (dd/mm/aaaa): ")
data_volta = input("🛬 Data de volta? (dd/mm/aaaa): ")
estilo = input("🧳 Estilo (econômico, conforto, aventura): ").lower()
orcamento = input("💰 Orçamento diário (R$): ")
lazer = input("🎭 Incluir shows/cultura local? (sim/não): ").lower()
pontos_turisticos = input("📌 Pontos turísticos específicos (vírgula) ou 'não': ")

try:
    orcamento_float = float(orcamento.replace(",", "."))
except ValueError:
    orcamento_float = 100.0

preferencias = (
    f"Ida: {data_ida}, Volta: {data_volta}, Estilo: {estilo}, "
    f"Orçamento: R${orcamento}/dia, Lazer: {lazer}."
)

print("\n🔆 Previsão do clima e câmbio...")
clima = obter_previsao_media(destino, data_ida, data_volta)
display(Markdown(clima))

print("\n✈️ Cotando preço da passagem aérea...")
voo = agente_voo(destino, cidade_origem, data_ida, data_volta)
display(Markdown(voo))

print("\n🏨 Sugerindo hospedagens econômicas...")
acomodacoes = agente_acomodacao(destino, data_ida, data_volta, orcamento_float)
display(Markdown(acomodacoes))

print("\n📄 Documentação necessária...")
docs = agente_documentos(destino, cidade_origem)
display(Markdown(docs))

data_hoje = datetime.today().strftime("%d/%m/%Y")
base_roteiro = agente_buscador(destino, data_hoje)

print("\n🧭 Gerando roteiro personalizado...")
roteiro_planejado = agente_planejador(destino, preferencias, base_roteiro, pontos_turisticos)

roteiro_formatado = agente_estruturador(destino, roteiro_planejado)
roteiro_com_links = add_maps_links(roteiro_formatado, destino)

display(Markdown("\n## 📅 Roteiro final da viagem"))
display(Markdown(roteiro_com_links))

print("\n🎉 Pronto! Seu roteiro está no ponto. Agora é só embarcar nessa viagem incrível! ✈️🌍")



